In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn import set_config
set_config(display="diagram")
from sklearn.naive_bayes import ComplementNB
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import plot_roc_curve, roc_auc_score,plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import plot_roc_curve, auc,roc_curve,roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.impute import SimpleImputer


In [2]:
df = pd.read_csv('/Users/mahmoud/Car_Price_Prediction/complete_car_data.csv')

In [3]:
df

,Car Name,Mileage,Dealer Name,Rating,Review Count,Price,Badge Label,Car Rating,Car Reviews,Accidents or damage,...,Open recall,Exterior color,Interior color,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Stock #
0,2016 Audi A4 2.0T Premium,"92,124 mi.",NJ State Auto Used Cars,4.2,(672 reviews),"$14,359",Good Deal,4.2,"['April 22, 2021\nBy Tommy from San Antonio tx...",None reported,...,Not available,Glacier White Metallic,Black,All-wheel Drive,21–30\n\n\n\nBased on EPA mileage ratings. Use...,E85 Flex Fuel,8-Speed Automatic,2.0L I4 16V GDI DOHC Turbo Flexible Fuel,WAUBFAFLXGN009815,48142
1,2015 Buick Enclave Leather,"111,488 mi.",NJ State Auto Used Cars,4.2,(672 reviews),"$13,517",Fair Deal,4.2,"['March 26, 2021\nBy Leo Appreciates a great r...",At least 1 accident or damage reported,...,At least 1 open recall reported,Iridium Metallic,Light Titanium,Front-wheel Drive,17–24\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,3.6L V6 24V GDI DOHC,5GAKRBKD6FJ125566,48175
2,2012 Jeep Grand Cherokee Laredo,"133,821 mi.",Road Masters II Inc.,NaN,(77 reviews),"$9,995",Great Deal | $399 under,4.5,"['June 28, 2023\nBy CatTayl from Chatsworth, I...",None reported,...,Not available,Brilliant Black Crystal Pearlcoat,Black,Four-wheel Drive,16–23\n\n\n\nBased on EPA mileage ratings. Use...,E85 Flex Fuel,Automatic,"3.6L V-6 DOHC, variable valve control, engine ...",1C4RJFAGXCC127988,4831
3,2010 Toyota Camry LE,"116,643 mi.",Arrow Motors,4.8,(220 reviews),"$9,999",Good Deal,5.0,"['January 24, 2023\nBy Pauline from St bernard...",At least 1 accident or damage reported,...,Not available,Green,Bisque,Front-wheel Drive,22–32\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,2.5L I4 16V MPFI DOHC,4T1BF3EK9AU561578,AU561578
4,2016 Mazda Mazda6 i Touring,"111,588 mi.",Arrow Motors,4.8,(220 reviews),"$13,499",Good Deal,5.0,"['October 30, 2022\nBy RChico456 from Las Vega...",At least 1 accident or damage reported,...,Not available,Snowflake White Pearl,Black,Front-wheel Drive,25–37\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Manual,2.5L I4 16V GDI DOHC,JM1GJ1V52G1488572,G1488572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9802,2021 Mitsubishi Mirage ES,"15,035 mi.",CarVision of Maple Shade,NaN,NaN,"$13,999",Fair Deal,4.9,"['April 18, 2021\nBy LackeyMO from Neosho, Mis...",At least 1 accident or damage reported,...,Not available,Blue,Black,Front-wheel Drive,33–41\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,5-Speed Manual,1.2L I3 12V MPFI DOHC,ML32AUHJ7MH010688,010688
9803,2006 Toyota Corolla S,"154,352 mi.",Bethlehem Ford,NaN,NaN,"$6,524",Great Deal | $194 under,5.0,"['April 27, 2023\nBy Kc from Whittier, MA\nUse...",At least 1 accident or damage reported,...,Not available,Silver Streak Mica,Dark Charcoal,Front-wheel Drive,30–38\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,4-Speed Automatic,1.8L I4 16V MPFI DOHC,2T1BR32E16C696228,KBF2413
9804,2008 Suzuki SX4 Base (A4),"46,460 mi.",Napoli Kia,NaN,NaN,"$8,924",Great Deal | $508 under,4.9,"['July 2, 2023\nBy Lee from Dayton ohio \nOwn...",None reported,...,Not available,Silver,–,All-wheel Drive,21–28\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,4-Speed Automatic,2.0L I4 16V MPFI DOHC,JS2YB413385100353,F5618
9805,2014 Hyundai Santa Fe Sport 2.0L Turbo,"152,296 mi.",Pine Belt Chevy of Lakewood,NaN,NaN,"$8,643",Good Deal,5.0,"['June 10, 2021\nBy Adelaide from NC\nOwns thi...",At least 1 accident or damage reported,...,Not available,Mineral Gray,Gray,Front-wheel Drive,19–27\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,5XYZU3LA5EG187545,L187545T


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9807 entries, 0 to 9806
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Car Name             9807 non-null   object 
 1   Mileage              9807 non-null   object 
 2   Dealer Name          9026 non-null   object 
 3   Rating               3900 non-null   float64
 4   Review Count         6011 non-null   object 
 5   Price                9807 non-null   object 
 6   Badge Label          7796 non-null   object 
 7   Car Rating           9807 non-null   object 
 8   Car Reviews          9807 non-null   object 
 9   Accidents or damage  9807 non-null   object 
 10  1-owner vehicle      9807 non-null   object 
 11  Personal use only    9807 non-null   object 
 12  Open recall          9807 non-null   object 
 13  Exterior color       9807 non-null   object 
 14  Interior color       9807 non-null   object 
 15  Drivetrain           9807 non-null   o

 0   Car Name             9807 non-null   object 
 1   Mileage              9807 non-null   object 
 2   Dealer Name          9026 non-null   object 
 3   Rating               3900 non-null   float64
 4   Review Count         6011 non-null   object 
 5   Price                9807 non-null   object 
 6   Badge Label          7796 non-null   object 
 7   Car Rating           9807 non-null   object 
 8   Car Reviews          9807 non-null   object 
 9   Accidents or damage  9807 non-null   object 
 10  1-owner vehicle      9807 non-null   object 
 11  Personal use only    9807 non-null   object 
 12  Open recall          9807 non-null   object 
 13  Exterior color       9807 non-null   object 
 14  Interior color       9807 non-null   object 
 15  Drivetrain           9807 non-null   object 
 16  MPG                  9021 non-null   object 
 17  Fuel type            9800 non-null   object 
 18  Transmission         9807 non-null   object 
 19  Engine               9807 non-null   object 
 20  VIN                  9807 non-null   object 
 21  Stock #              9026 non-null   object 

In [6]:
# getting year make model type from name
df[['Year', 'Make', 'Model', 'Type']] = df['Car Name'].str.extract(r'(\d{4})\s+([^ ]+)\s+([^ ]+)\s+(.*)')

In [7]:
# Having the mileage column in float rather than string
df['Mileage'] = df['Mileage'].str.replace(' mi.', '').str.replace(',', '')
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')

0        92124.0
1       111488.0
2       133821.0
3       116643.0
4       111588.0
          ...   
9802     15035.0
9803    154352.0
9804     46460.0
9805    152296.0
9806    124233.0
Name: Mileage, Length: 9807, dtype: float64

In [8]:
# Since Price is a string, lets convert to a float

# First remove dollar sign
df['Price'] = df['Price'].str.replace('[^\d.]', '', regex=True)

# Now make it a float
df['Price'] = df['Price'].astype(float)

# Confirm the updated data type
df['Price'].dtype

dtype('float64')

In [9]:
# rating to dealer rating
df = df.rename(columns={"Rating": "Dealer Rating"})

In [10]:
df.isna().sum()

Car Name                  0
Mileage                   5
Dealer Name             781
Dealer Rating          5907
Review Count           3796
Price                     0
Badge Label            2011
Car Rating                0
Car Reviews               0
Accidents or damage       0
1-owner vehicle           0
Personal use only         0
Open recall               0
Exterior color            0
Interior color            0
Drivetrain                0
MPG                     786
Fuel type                 7
Transmission              0
Engine                    0
VIN                       0
Stock #                 781
Year                      0
Make                      0
Model                     0
Type                      0
dtype: int64

In [11]:
# NO need for stock # so get rid of it
df.drop(['Stock #'], axis =1, inplace=True)

# Nans
- Mileage                   5
- Dealer Name             781
- Dealer Rating          5907
- Badge Label            2011
- MPG                     786
- Fuel type                 7
- Stock #                 781

In [13]:
df

,Car Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Badge Label,Car Rating,Car Reviews,Accidents or damage,...,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Year,Make,Model,Type
0,2016 Audi A4 2.0T Premium,92124.0,NJ State Auto Used Cars,4.2,(672 reviews),14359.0,Good Deal,4.2,"['April 22, 2021\nBy Tommy from San Antonio tx...",None reported,...,All-wheel Drive,21–30\n\n\n\nBased on EPA mileage ratings. Use...,E85 Flex Fuel,8-Speed Automatic,2.0L I4 16V GDI DOHC Turbo Flexible Fuel,WAUBFAFLXGN009815,2016,Audi,A4,2.0T Premium
1,2015 Buick Enclave Leather,111488.0,NJ State Auto Used Cars,4.2,(672 reviews),13517.0,Fair Deal,4.2,"['March 26, 2021\nBy Leo Appreciates a great r...",At least 1 accident or damage reported,...,Front-wheel Drive,17–24\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,3.6L V6 24V GDI DOHC,5GAKRBKD6FJ125566,2015,Buick,Enclave,Leather
2,2012 Jeep Grand Cherokee Laredo,133821.0,Road Masters II Inc.,NaN,(77 reviews),9995.0,Great Deal | $399 under,4.5,"['June 28, 2023\nBy CatTayl from Chatsworth, I...",None reported,...,Four-wheel Drive,16–23\n\n\n\nBased on EPA mileage ratings. Use...,E85 Flex Fuel,Automatic,"3.6L V-6 DOHC, variable valve control, engine ...",1C4RJFAGXCC127988,2012,Jeep,Grand,Cherokee Laredo
3,2010 Toyota Camry LE,116643.0,Arrow Motors,4.8,(220 reviews),9999.0,Good Deal,5.0,"['January 24, 2023\nBy Pauline from St bernard...",At least 1 accident or damage reported,...,Front-wheel Drive,22–32\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,2.5L I4 16V MPFI DOHC,4T1BF3EK9AU561578,2010,Toyota,Camry,LE
4,2016 Mazda Mazda6 i Touring,111588.0,Arrow Motors,4.8,(220 reviews),13499.0,Good Deal,5.0,"['October 30, 2022\nBy RChico456 from Las Vega...",At least 1 accident or damage reported,...,Front-wheel Drive,25–37\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Manual,2.5L I4 16V GDI DOHC,JM1GJ1V52G1488572,2016,Mazda,Mazda6,i Touring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9802,2021 Mitsubishi Mirage ES,15035.0,CarVision of Maple Shade,NaN,NaN,13999.0,Fair Deal,4.9,"['April 18, 2021\nBy LackeyMO from Neosho, Mis...",At least 1 accident or damage reported,...,Front-wheel Drive,33–41\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,5-Speed Manual,1.2L I3 12V MPFI DOHC,ML32AUHJ7MH010688,2021,Mitsubishi,Mirage,ES
9803,2006 Toyota Corolla S,154352.0,Bethlehem Ford,NaN,NaN,6524.0,Great Deal | $194 under,5.0,"['April 27, 2023\nBy Kc from Whittier, MA\nUse...",At least 1 accident or damage reported,...,Front-wheel Drive,30–38\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,4-Speed Automatic,1.8L I4 16V MPFI DOHC,2T1BR32E16C696228,2006,Toyota,Corolla,S
9804,2008 Suzuki SX4 Base (A4),46460.0,Napoli Kia,NaN,NaN,8924.0,Great Deal | $508 under,4.9,"['July 2, 2023\nBy Lee from Dayton ohio \nOwn...",None reported,...,All-wheel Drive,21–28\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,4-Speed Automatic,2.0L I4 16V MPFI DOHC,JS2YB413385100353,2008,Suzuki,SX4,Base (A4)
9805,2014 Hyundai Santa Fe Sport 2.0L Turbo,152296.0,Pine Belt Chevy of Lakewood,NaN,NaN,8643.0,Good Deal,5.0,"['June 10, 2021\nBy Adelaide from NC\nOwns thi...",At least 1 accident or damage reported,...,Front-wheel Drive,19–27\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,5XYZU3LA5EG187545,2014,Hyundai,Santa,Fe Sport 2.0L Turbo


In [16]:
# having review count become float 
# df['Review Count'] = df['Review Count'].str.extract('(\d+)').astype(float)

# For badge label we will have the type of deal on a scale of 1-5

In [20]:
df["Car Rating"].unique()

array(['4.2', '4.5', '5.0', '3.8', '4.6', '4.3', '3.1', '4.8', '3.2',
       '4.7', '3.0', '3.6', '4.4', '3.9', '4.9', '3.4', '3.5', '4.0',
       'No rating available', '4.1', '2.4', '2.7', '1.2', '2.1', '2.9',
       '1.9', '2.6', '2.2', '3.7', '2.0', '1.6', '1.7', '1.5', '2.5',
       '1.0', '3.3'], dtype=object)

In [22]:
# So we have 'No rating available' we will replace that with the mean
df["Car Rating"] = pd.to_numeric(df["Car Rating"], errors="coerce")
mean_rating = df["Car Rating"].dropna().mean()
df["Car Rating"].fillna(mean_rating, inplace=True)

df["Car Rating"].dtype

dtype('float64')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9807 entries, 0 to 9806
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Car Name             9807 non-null   object 
 1   Mileage              9802 non-null   float64
 2   Dealer Name          9026 non-null   object 
 3   Dealer Rating        3900 non-null   float64
 4   Review Count         6011 non-null   float64
 5   Price                9807 non-null   float64
 6   Badge Label          7796 non-null   object 
 7   Car Rating           9807 non-null   object 
 8   Car Reviews          9807 non-null   object 
 9   Accidents or damage  9807 non-null   object 
 10  1-owner vehicle      9807 non-null   object 
 11  Personal use only    9807 non-null   object 
 12  Open recall          9807 non-null   object 
 13  Exterior color       9807 non-null   object 
 14  Interior color       9807 non-null   object 
 15  Drivetrain           9807 non-null   o

In [23]:
df['1-owner vehicle']

0        No
1       Yes
2        No
3       Yes
4        No
       ... 
9802     No
9803     No
9804     No
9805     No
9806    Yes
Name: 1-owner vehicle, Length: 9807, dtype: object

# We will also have the above column if no = 0 and yes = 1

In [24]:
df['Personal use only']

0       Yes
1       Yes
2        No
3       Yes
4       Yes
       ... 
9802     No
9803    Yes
9804    Yes
9805    Yes
9806    Yes
Name: Personal use only, Length: 9807, dtype: object

# Same here, Personal Use only

In [25]:
df['Open recall']

0                         Not available
1       At least 1 open recall reported
2                         Not available
3                         Not available
4                         Not available
                     ...               
9802                      Not available
9803                      Not available
9804                      Not available
9805                      Not available
9806                      Not available
Name: Open recall, Length: 9807, dtype: object

In [26]:
df["Open recall"].unique()

array(['Not available', 'At least 1 open recall reported'], dtype=object)

# Same here, Not available = 0, At leat 1 ope recall reported = 1

In [27]:
df['MPG']

0       21–30\n\n\n\nBased on EPA mileage ratings. Use...
1       17–24\n\n\n\nBased on EPA mileage ratings. Use...
2       16–23\n\n\n\nBased on EPA mileage ratings. Use...
3       22–32\n\n\n\nBased on EPA mileage ratings. Use...
4       25–37\n\n\n\nBased on EPA mileage ratings. Use...
                              ...                        
9802    33–41\n\n\n\nBased on EPA mileage ratings. Use...
9803    30–38\n\n\n\nBased on EPA mileage ratings. Use...
9804    21–28\n\n\n\nBased on EPA mileage ratings. Use...
9805    19–27\n\n\n\nBased on EPA mileage ratings. Use...
9806    19–28\n\n\n\nBased on EPA mileage ratings. Use...
Name: MPG, Length: 9807, dtype: object

In [29]:
df['MPG'] = df['MPG'].str.extract(r'(\d+)').astype(float)
df['MPG'].dtype

dtype('float64')

In [30]:
df['MPG']

0       21.0
1       17.0
2       16.0
3       22.0
4       25.0
        ... 
9802    33.0
9803    30.0
9804    21.0
9805    19.0
9806    19.0
Name: MPG, Length: 9807, dtype: float64

In [31]:
# we can fill mpg with the average mpg which is 25.4 (from google)
df['MPG'] = df['MPG'].fillna(25.4)

In [32]:
df.isna().sum()

Car Name                  0
Mileage                   5
Dealer Name             781
Dealer Rating          5907
Review Count           3796
Price                     0
Badge Label            2011
Car Rating                0
Car Reviews               0
Accidents or damage       0
1-owner vehicle           0
Personal use only         0
Open recall               0
Exterior color            0
Interior color            0
Drivetrain                0
MPG                       0
Fuel type                 7
Transmission              0
Engine                    0
VIN                       0
Year                      0
Make                      0
Model                     0
Type                      0
dtype: int64

In [33]:
df['Drivetrain']

0         All-wheel Drive
1       Front-wheel Drive
2        Four-wheel Drive
3       Front-wheel Drive
4       Front-wheel Drive
              ...        
9802    Front-wheel Drive
9803    Front-wheel Drive
9804      All-wheel Drive
9805    Front-wheel Drive
9806    Front-wheel Drive
Name: Drivetrain, Length: 9807, dtype: object

# We will one hot encode Drivetrain

In [34]:
df['Fuel type']

0       E85 Flex Fuel
1            Gasoline
2       E85 Flex Fuel
3            Gasoline
4            Gasoline
            ...      
9802         Gasoline
9803         Gasoline
9804         Gasoline
9805         Gasoline
9806         Gasoline
Name: Fuel type, Length: 9807, dtype: object

In [35]:
df[df["Fuel type"].isna()]

,Car Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Badge Label,Car Rating,Car Reviews,Accidents or damage,...,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Year,Make,Model,Type
377,2015 BMW i3 Base,35759.0,Car Revolution,2.6,497.0,13881.0,Fair Deal,2.6,"['March 1, 2021\nBy Carfan101 from Miami\nOwn...",None reported,...,Rear-wheel Drive,25.4,NaN,1-Speed Automatic,Electric,WBY1Z2C51FV555501,2015,BMW,i3,Base
715,2015 BMW i3 Base,24270.0,Car Revolution,2.6,497.0,12946.0,Good Deal,2.6,"['March 1, 2021\nBy Carfan101 from Miami\nOwn...",At least 1 accident or damage reported,...,Rear-wheel Drive,25.4,NaN,1-Speed Automatic,Electric,WBY1Z2C56FV555557,2015,BMW,i3,Base
1182,2015 Chevrolet Volt Base,96087.0,Brick City Motors,3.0,85.0,12995.0,"Great Deal | $2,244 under",3.0,"['February 1, 2020\nBy Jorge from San Diego, ...",At least 1 accident or damage reported,...,Front-wheel Drive,25.4,NaN,1-Speed Automatic,Electric,1G1RC6E4XFU121500,2015,Chevrolet,Volt,Base
1620,2018 Nissan Leaf SV,66660.0,GTR Auto Solutions,NaN,1.0,14900.0,Hot Car,4.6,"['April 2, 2022\nBy DAVE BLAKMON from SEABROO...",None reported,...,Front-wheel Drive,25.4,NaN,Automatic,Electric,1N4AZ1CP3JC307788,2018,Nissan,Leaf,SV
1822,2017 FIAT 500e Battery Electric,33812.0,NaN,NaN,NaN,14999.0,Fair Deal,4.5,"['June 17, 2021\nBy HB Surfer from Huntington ...",None reported,...,Front-wheel Drive,25.4,NaN,1-Speed A/T,111.0HP Electric Motor Electric Fuel System,3C3CFFGE9HT546055,2017,FIAT,500e,Battery Electric
2075,2012 Nissan Leaf SL,26930.0,NaN,NaN,NaN,6999.0,NaN,4.3,"['April 11, 2021\nBy Jimmy from Clovis\nOwns t...",None reported,...,Front-wheel Drive,25.4,NaN,1-Speed A/T,107.0HP Electric Motor Electric Fuel System,JN1AZ0CP4CT016002,2012,Nissan,Leaf,SL
2079,2018 Nissan Leaf SV,63594.0,Route 46 Nissan,3.7,1.0,12995.0,Fair Deal,4.4,"['April 2, 2022\nBy DAVE BLAKMON from SEABROO...",None reported,...,Front-wheel Drive,25.4,NaN,1-Speed Automatic,Electric,1N4AZ1CP2JC302730,2018,Nissan,Leaf,SV


# They all take Electricity 

In [36]:
# we can fill fuel type with Electricity
df['Fuel type'] = df['Fuel type'].fillna('Electricity')

In [37]:
df.isna().sum()

Car Name                  0
Mileage                   5
Dealer Name             781
Dealer Rating          5907
Review Count           3796
Price                     0
Badge Label            2011
Car Rating                0
Car Reviews               0
Accidents or damage       0
1-owner vehicle           0
Personal use only         0
Open recall               0
Exterior color            0
Interior color            0
Drivetrain                0
MPG                       0
Fuel type                 0
Transmission              0
Engine                    0
VIN                       0
Year                      0
Make                      0
Model                     0
Type                      0
dtype: int64

In [38]:
df[df["Mileage"].isna()]

,Car Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Badge Label,Car Rating,Car Reviews,Accidents or damage,...,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Year,Make,Model,Type
3427,2014 Dodge Durango SXT,NaN,MajorWorld Chevy,3.5,2.0,8995.0,NaN,3.5,"['January 23, 2023\nBy Edward G Jones from Dur...",At least 1 accident or damage reported,...,All-wheel Drive,17.0,E85 Flex Fuel,8-Speed Automatic,3.6L V6 24V MPFI DOHC Flexible Fuel,1C4RDJAG7EC318796,2014,Dodge,Durango,SXT
3438,2015 Toyota Camry SE,NaN,MajorWorld Chevy,3.5,2.0,12588.0,NaN,3.5,"['April 19, 2022\nBy Jackson Porter from Provo...",At least 1 accident or damage reported,...,Front-wheel Drive,25.0,Gasoline,6-Speed Automatic,2.5L I4 16V MPFI DOHC,4T1BF1FK8FU111172,2015,Toyota,Camry,SE
3441,2017 Ford Fusion SE,NaN,MajorWorld Chevy,3.5,2.0,7995.0,NaN,3.5,"['April 25, 2023\nBy Ruth P. Turner from haysi...",At least 1 accident or damage reported,...,Front-wheel Drive,21.0,Gasoline,6-Speed Automatic,1.5L I4 16V GDI DOHC Turbo,3FA6P0HD6HR298312,2017,Ford,Fusion,SE
3447,2006 Honda Civic EX,NaN,MajorWorld Chevy,3.5,2.0,5888.0,NaN,3.5,"['June 22, 2022\nBy Ken from Portland, Oregon\...",At least 1 accident or damage reported,...,Front-wheel Drive,30.0,Gasoline,5-Speed Automatic,1.8L I4 16V MPFI SOHC,2HGFA16876H507820,2006,Honda,Civic,EX
4039,2017 Honda Accord EX-L,NaN,Victory Mitsubishi,4.1,1.0,13995.0,NaN,4.9,"['April 24, 2023\nBy Vera from Parma, OH\nOwns...",None reported,...,Front-wheel Drive,21.0,Gasoline,6-Speed Automatic,3.5L V6 24V MPFI SOHC,1HGCR3F84HA037707,2017,Honda,Accord,EX-L


In [39]:
# dropping the mileage Nans
df = df.dropna(subset=['Mileage'])

# Remaining Nans
- Dealer Name             781
- Dealer Rating          5907
- Review Count           3796
- Badge Label            2011

In [40]:
# Replacing Dealer Name with Unstated
df['Dealer Name'] = df['Dealer Name'].fillna('Unstated')

<ipython-input-40-66243eec8f27>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dealer Name'] = df['Dealer Name'].fillna('Unstated')


In [41]:
# All Nans in rating to the mean
rating_mean = df['Dealer Rating'].mean()
df['Dealer Rating'].fillna(rating_mean, inplace=True)

/Users/mahmoud/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [42]:
# So we can fill those counts with 0 as well
df['Review Count'] = df['Review Count'].fillna(0)

<ipython-input-42-1edc1a72ac90>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Review Count'] = df['Review Count'].fillna(0)


In [44]:
df[df["Badge Label"].isna()]

,Car Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Badge Label,Car Rating,Car Reviews,Accidents or damage,...,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Year,Make,Model,Type
13,2021 Chevrolet Equinox 1LT,57237.0,Auto Lux,3.800000,79.0,14439.0,NaN,3.8,"['May 19, 2023\nBy Cathy sutts from Davie fl\n...",None reported,...,All-wheel Drive,25.0,Gasoline,6-Speed Automatic,1.5L I4 16V GDI DOHC Turbo,2GNAXUEV3M6113084,2021,Chevrolet,Equinox,1LT
18,2012 Jeep Liberty Sport,127370.0,Euro Motor Haus,3.100000,58.0,5894.0,NaN,3.1,"['May 7, 2023\nBy Emmanuel Alalibo Gouldekek...",None reported,...,Four-wheel Drive,15.0,Gasoline,Automatic,Power Tech 3.7L V-6 engine with 210HP,1C4PJMAK5CW137987,2012,Jeep,Liberty,Sport
23,1989 Mazda RX-7,90462.0,Brick City Motors,3.000000,85.0,7995.0,NaN,3.0,[],None reported,...,Rear-wheel Drive,25.4,Gasoline,Automatic,1.3L Rotary,JM1FC3510K0708787,1989,Mazda,RX-7,
40,2006 BMW 525 i,86300.0,Unstated,3.849628,0.0,7500.0,NaN,4.6,"['August 12, 2022\nBy Luis R. from Mexico\nOwn...",None reported,...,Rear-wheel Drive,25.4,Gasoline,6-Speed A/T,215.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,WBANE53586CK83196,2006,BMW,525,i
43,2009 MINI Cooper Clubman,78735.0,Keyless Motors,4.700000,194.0,4998.0,NaN,4.7,"['August 27, 2020\nBy Disappointed and Strande...",None reported,...,Front-wheel Drive,28.0,Gasoline,6-Speed Manual,1.6L I4 16V MPFI DOHC,WMWML33509TP98381,2009,MINI,Cooper,Clubman
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9792,2005 Lincoln Aviator Luxury,140000.0,Unstated,3.849628,0.0,2000.0,NaN,4.8,"['June 8, 2022\nBy Rit-C from Auburn NH\nOwns...",None reported,...,All-wheel Drive,25.4,Gasoline,A/T,302.0HP 4.6L 8 Cylinder Engine Gasoline Fuel,5LMEU88H75ZJ17847,2005,Lincoln,Aviator,Luxury
9793,2005 Saturn L 300,105700.0,Unstated,3.849628,0.0,2800.0,NaN,3.7,"['June 27, 2023\nBy LadyMadeline from Leavenwo...",None reported,...,Front-wheel Drive,25.4,Gasoline,A/T,182.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,1G8JD54RX5Y503844,2005,Saturn,L,300
9794,2017 Ford Fusion SE,101472.0,Tuckahoe Autoland,3.849628,2.0,11495.0,NaN,4.8,"['April 25, 2023\nBy Ruth P. Turner from haysi...",None reported,...,Front-wheel Drive,21.0,Gasoline,Automatic,"Duratec 2.5L I-4 DOHC, Ti-VCT variable valve c...",3FA6P0H75HR198129,2017,Ford,Fusion,SE
9795,2013 BMW X5 xDrive50i,93307.0,Tuckahoe Autoland,3.849628,2.0,12995.0,NaN,4.6,"['November 23, 2022\nBy Larry from Gastonia N...",At least 1 accident or damage reported,...,All-wheel Drive,14.0,Gasoline,Automatic,"V8, Twin Turbo, 4.4 Liter",5UXZV8C52D0C16907,2013,BMW,X5,xDrive50i


In [47]:
df["Badge Label"].unique()

array(['Good Deal', 'Fair Deal', 'Great Deal | $399 under', ...,
       'Great Deal | $1,270 under', 'Great Deal | $1,440 under',
       'Good Deal | $162 under'], dtype=object)

In [51]:
df['Badge Label']

0                     Good Deal
1                     Fair Deal
2       Great Deal | $399 under
3                     Good Deal
4                     Good Deal
                 ...           
9802                  Fair Deal
9803    Great Deal | $194 under
9804    Great Deal | $508 under
9805                  Good Deal
9806                  Fair Deal
Name: Badge Label, Length: 9802, dtype: object

# It seems like there is no 'Bad Deal'

In [48]:
# So we can fill those Nans with 'Bad Deal' 
df['Badge Label'] = df['Badge Label'].fillna('Bad Deal')

<ipython-input-48-99334d54244e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Badge Label'] = df['Badge Label'].fillna('Bad Deal')


In [50]:
df.isna().sum()

Car Name               0
Mileage                0
Dealer Name            0
Dealer Rating          0
Review Count           0
Price                  0
Badge Label            0
Car Rating             0
Car Reviews            0
Accidents or damage    0
1-owner vehicle        0
Personal use only      0
Open recall            0
Exterior color         0
Interior color         0
Drivetrain             0
MPG                    0
Fuel type              0
Transmission           0
Engine                 0
VIN                    0
Year                   0
Make                   0
Model                  0
Type                   0
dtype: int64

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9802 entries, 0 to 9806
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Car Name             9802 non-null   object 
 1   Mileage              9802 non-null   float64
 2   Dealer Name          9802 non-null   object 
 3   Dealer Rating        9802 non-null   float64
 4   Review Count         9802 non-null   float64
 5   Price                9802 non-null   float64
 6   Badge Label          9802 non-null   object 
 7   Car Rating           9802 non-null   float64
 8   Car Reviews          9802 non-null   object 
 9   Accidents or damage  9802 non-null   object 
 10  1-owner vehicle      9802 non-null   object 
 11  Personal use only    9802 non-null   object 
 12  Open recall          9802 non-null   object 
 13  Exterior color       9802 non-null   object 
 14  Interior color       9802 non-null   object 
 15  Drivetrain           9802 non-null   o

# With badge label- we can either remove 399 under (when its a Great Deal) or one hot encode as is 

# We will also have the 1-owner vehicle column if no = 0 and yes = 1
# For badge label we will have the type of deal on a scale 
# Same here, Personal Use only
# Same here, Not available = 0, At leat 1 ope recall reported = 1
# We will one hot encode Drivetrain
# We will one hot encode Exterior and Interior Color
# Drivetrain one hot encoded as well
# Fuel type one hot encoded

In [53]:
df['Exterior color']

0                  Glacier White Metallic
1                        Iridium Metallic
2       Brilliant Black Crystal Pearlcoat
3                                   Green
4                   Snowflake White Pearl
                      ...                
9802                                 Blue
9803                   Silver Streak Mica
9804                               Silver
9805                         Mineral Gray
9806                Modern Steel Metallic
Name: Exterior color, Length: 9802, dtype: object

In [54]:
df['Drivetrain']

0         All-wheel Drive
1       Front-wheel Drive
2        Four-wheel Drive
3       Front-wheel Drive
4       Front-wheel Drive
              ...        
9802    Front-wheel Drive
9803    Front-wheel Drive
9804      All-wheel Drive
9805    Front-wheel Drive
9806    Front-wheel Drive
Name: Drivetrain, Length: 9802, dtype: object

In [56]:
df['Fuel type'].unique()

array(['E85 Flex Fuel', 'Gasoline', 'Hybrid', 'Diesel', '–',
       'Electricity', 'Regular Unleaded', 'Flexible Fuel'], dtype=object)

In [57]:
# Which rows have that weird symbol
df[df["Fuel type"].str.contains('–', na=False)]

,Car Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Badge Label,Car Rating,Car Reviews,Accidents or damage,...,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Year,Make,Model,Type
258,1995 Mazda MX-5 Miata Base,115000.0,Unstated,3.849628,0.0,12000.0,Great Deal | $493 under,4.400000,"['April 27, 2023\nBy Dennis Wild from Elisabet...",None reported,...,Rear-wheel Drive,25.4,–,M/T,–,JM1NA3539S0610005,1995,Mazda,MX-5,Miata Base
291,2012 Nissan Titan SV,167113.0,Auto Spot LLC,3.400000,228.0,10999.0,Good Deal,3.400000,"['July 17, 2022\nBy Howie from Chicago Suburbs...",None reported,...,Four-wheel Drive,12.0,–,5-Speed Automatic,5.6L V8 32V MPFI DOHC,1N6AA0EC7CN311711,2012,Nissan,Titan,SV
528,2014 Mercedes-Benz E-Class E 350,137722.0,Magic Auto Sales,2.700000,197.0,11990.0,"Great Deal | $1,262 under",2.700000,"['June 12, 2023\nBy Latrsss from Tupelo Missis...",At least 1 accident or damage reported,...,Rear-wheel Drive,21.0,–,7-Speed Automatic,3.5L V6 24V GDI DOHC,WDDKJ5KB5EF241743,2014,Mercedes-Benz,E-Class,E 350
678,2010 Ford E350 Super Duty XLT,162594.0,Fast Track Motors,4.500000,222.0,11885.0,Bad Deal,5.000000,"['November 16, 2022\nBy just a guy from athens...",Not available,...,–,25.4,–,Automatic,0.0,IFDSE3FL5ADA54106,2010,Ford,E350,Super Duty XLT
800,2013 RAM Cargo Tradesman,141837.0,KBB Auto Sales,4.900000,73.0,6500.0,Fair Deal,4.900000,"['May 15, 2023\nBy Chris from Marietta georgia...",None reported,...,Front-wheel Drive,17.0,–,Automatic,3.6L V6 24V MPFI DOHC,2C4RRGAGXDR587501,2013,RAM,Cargo,Tradesman
1082,2015 Chrysler Town & Country Touring,105912.0,Dash Auto Gallery Inc,4.800000,81.0,13491.0,Fair Deal,4.800000,"['July 1, 2022\nBy fleetfoot from North Verno...",None reported,...,Front-wheel Drive,17.0,–,Automatic,3.6L V6 24V MPFI DOHC,2C4RC1BG9FR627809,2015,Chrysler,Town,& Country Touring
1187,2014 Chrysler 300 S,114750.0,Queens Auto Mall,4.000000,2.0,14980.0,Good Deal,4.000000,"['August 2, 2022\nBy thefoot524 from Suffolk,...",At least 1 accident or damage reported,...,All-wheel Drive,18.0,–,Automatic,3.6L V6 24V MPFI DOHC,2C3CCAGG7EH179643,2014,Chrysler,300,S
1460,2013 Chrysler Town & Country Touring,101150.0,Brick City Motors,3.000000,85.0,10500.0,Good Deal,3.000000,"['May 24, 2022\nBy Claudette from Cache ok\nO...",None reported,...,Front-wheel Drive,17.0,–,Automatic,3.6L V6 24V MPFI DOHC,2C4RC1BG5DR777784,2013,Chrysler,Town,& Country Touring
1466,2015 RAM Cargo Tradesman,170829.0,Auto Spot LLC,3.400000,228.0,5199.0,Good Deal,3.400000,[],At least 1 accident or damage reported,...,Front-wheel Drive,18.0,–,Automatic,3.6L V6 24V MPFI DOHC,2C4JRGAGXFR581505,2015,RAM,Cargo,Tradesman
1471,2015 Chrysler Town & Country Touring,76437.0,Auto Spot LLC,3.400000,228.0,12199.0,Good Deal | $209 under,3.400000,"['July 1, 2022\nBy fleetfoot from North Verno...",At least 1 accident or damage reported,...,Front-wheel Drive,17.0,–,Automatic,3.6L V6 24V MPFI DOHC,2C4RC1BG7FR626755,2015,Chrysler,Town,& Country Touring


In [58]:
# Replace '–' with 'Gasoline'
df["Fuel type"] = df["Fuel type"].replace('–', 'Gasoline')
df["Fuel type"].unique()

<ipython-input-58-5aad9846dfaf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Fuel type"] = df["Fuel type"].replace('–', 'Gasoline')


array(['E85 Flex Fuel', 'Gasoline', 'Hybrid', 'Diesel', 'Electricity',
       'Regular Unleaded', 'Flexible Fuel'], dtype=object)

In [59]:
df['Engine'].unique()

array(['2.0L I4 16V GDI DOHC Turbo Flexible Fuel', '3.6L V6 24V GDI DOHC',
       '3.6L V-6 DOHC, variable valve control, engine with 290HP',
       '2.5L I4 16V MPFI DOHC', '2.5L I4 16V GDI DOHC',
       '1.8L I4 16V GDI DOHC Turbo', '3.5L V6 24V MPFI SOHC',
       '3.7L V6 24V MPFI SOHC', '2.3L I4 16V MPFI DOHC',
       '3.5L V6 24V MPFI DOHC', '1.5L I4 16V GDI DOHC Turbo',
       '2.0L I4 16V GDI DOHC Turbo', '2.4L I4 16V GDI DOHC',
       '2.4L I4 16V MPFI DOHC', 'Power Tech 3.7L V-6 engine with 210HP',
       'Duratec 2.5L I-4 DOHC, Ti-VCT variable valve control, engine wit',
       '2L I-4 gasoline direct injection, DOHC, variable valve control,',
       '2.5L I-4 DOHC, variable valve control, engine with 182HP',
       '1.3L Rotary', '2.4L I4 16V MPFI SOHC', 'V6, 3.6 Liter',
       '2.5L H4 16V MPFI DOHC', '1.6L I4 16V GDI DOHC Turbo',
       '2.5L I5 20V MPFI DOHC',
       '2.7L H-6 DOHC, variable valve control, engine with 217HP',
       '5.0L V8 32V GDI DOHC', '1.4L I4 16V MP

In [60]:
# Which rows have that weird symbol
df[df["Engine"].str.contains('–', na=False)]

,Car Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Badge Label,Car Rating,Car Reviews,Accidents or damage,...,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Year,Make,Model,Type
258,1995 Mazda MX-5 Miata Base,115000.0,Unstated,3.849628,0.0,12000.0,Great Deal | $493 under,4.400000,"['April 27, 2023\nBy Dennis Wild from Elisabet...",None reported,...,Rear-wheel Drive,25.4,Gasoline,M/T,–,JM1NA3539S0610005,1995,Mazda,MX-5,Miata Base
1378,2014 RAM Cargo Tradesman,117697.0,Route 4 Auto Exchange,3.849628,9.0,6995.0,Bad Deal,4.500000,"['October 17, 2017\nBy Nick Nick Nick from Gar...",At least 1 accident or damage reported,...,Front-wheel Drive,18.0,Gasoline,Automatic,–,2C4JRGAGXER249255,2014,RAM,Cargo,Tradesman
1939,1996 Nissan Quest GXE,152000.0,Unstated,3.849628,0.0,2400.0,Bad Deal,4.400000,"['March 24, 2012\nBy Geri from Vermont & Flori...",None reported,...,Front-wheel Drive,25.4,Gasoline,A/T,–,4N2DN11W7TD838452,1996,Nissan,Quest,GXE
1961,2017 Honda Civic,77331.0,Plaza Auto Mall,4.000000,1.0,15000.0,Great Deal | $583 under,4.000000,"['June 23, 2023\nBy Wings from Detroit, MI \nO...",At least 1 accident or damage reported,...,–,25.4,Gasoline,Continuously Variable Transmission,–,19XFC2F58HE028597,2017,Honda,Civic,
2065,2012 Chrysler 200 Touring,148176.0,Unstated,3.849628,0.0,5999.0,Great Deal | $840 under,4.400000,"['June 28, 2023\nBy Chuck from Elk River,Mn\nO...",At least 1 accident or damage reported,...,Front-wheel Drive,25.4,Gasoline,6-Speed A/T,–,1C3CCBBB0CN138406,2012,Chrysler,200,Touring
2354,2012 Dodge Avenger SE,150000.0,Unstated,3.849628,0.0,4500.0,Bad Deal,4.300000,"['September 21, 2022\nBy Dreammr64 from Maryl...",None reported,...,Front-wheel Drive,25.4,Gasoline,A/T,–,1C3CDZAB4CN310623,2012,Dodge,Avenger,SE
2355,2008 Dodge Avenger SXT,130000.0,Unstated,3.849628,0.0,5700.0,"Great Deal | $1,126 under",4.200000,"['June 23, 2023\nBy TractorGuy11 from Owensvil...",None reported,...,Front-wheel Drive,25.4,Gasoline,A/T,–,1B3LC56J28N216188,2008,Dodge,Avenger,SXT
2380,1999 Ford Crown Victoria Base,74545.0,Atlantic Car Sales,3.849628,4.0,2999.0,Bad Deal,4.600000,"['February 25, 2020\nBy Whip cream from Norwel...",None reported,...,Rear-wheel Drive,17.0,Gasoline,Automatic,–,2FAFP73W9XX176139,1999,Ford,Crown,Victoria Base
2514,1994 Ford Aerostar XLT 4WD,211269.0,Unstated,3.849628,0.0,3000.0,Bad Deal,4.300000,"['December 18, 2020\nBy JamesRoss from Regina,...",None reported,...,Four-wheel Drive,25.4,Gasoline,A/T,–,1FMDA41X3RZA25951,1994,Ford,Aerostar,XLT 4WD
3271,1996 Toyota 4Runner SR5 4WD,231000.0,Unstated,3.849628,0.0,7500.0,Bad Deal,4.700000,"['May 1, 2023\nBy Monty makowsky from Big spr...",None reported,...,Four-wheel Drive,25.4,Gasoline,A/T,–,JT3HN86RXT0024033,1996,Toyota,4Runner,SR5 4WD


# So i also have that weird - in Drivetrain, Transmission and in Reviews I have []	

In [61]:
# Replace '–' with 'Unstated'
df["Engine"] = df["Engine"].replace('–', 'Unstated')

<ipython-input-61-f05a69f303b0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Engine"] = df["Engine"].replace('–', 'Unstated')


In [62]:
df['Transmission'] = df['Transmission'].replace('-', 'Unstated')

<ipython-input-62-26ca287f91c2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Transmission'] = df['Transmission'].replace('-', 'Unstated')


In [64]:
# Replace [] with Unknown 
df["Car Reviews"] = df["Car Reviews"].replace('[]', 'Unknown')

<ipython-input-64-7b0123922978>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Car Reviews"] = df["Car Reviews"].replace('[]', 'Unknown')


In [65]:
# we dont want how much under, just the type of deal
df["Badge Label"] = df["Badge Label"].str.split('|').str[0].str.strip()
df["Badge Label"].unique()

<ipython-input-65-ec6326d22ed8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Badge Label"] = df["Badge Label"].str.split('|').str[0].str.strip()


array(['Good Deal', 'Fair Deal', 'Great Deal', 'Bad Deal', 'Hot Car'],
      dtype=object)

In [66]:
# Whats Hot Car
df[df["Badge Label"] == 'Hot Car']

,Car Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Badge Label,Car Rating,Car Reviews,Accidents or damage,...,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Year,Make,Model,Type
838,2015 Chevrolet City Express 1LS,120425.0,Sho Case Auto,4.500000,28.0,10900.0,Hot Car,4.5,"[""March 19, 2021\nBy I'm so cold. from Ellswor...",None reported,...,Front-wheel Drive,24.0,Gasoline,Automatic CVT,2.0L I4 16V MPFI DOHC,3N63M0YN6FK703599,2015,Chevrolet,City,Express 1LS
1006,2014 RAM 1500 SLT,171084.0,Auto Spot LLC,3.400000,228.0,11499.0,Hot Car,3.4,"['July 12, 2023\nBy John O’Neil from Milford, ...",None reported,...,Four-wheel Drive,16.0,Diesel,8-Speed Automatic,3.0L V6 24V DDI DOHC Turbo Diesel,1C6RR7GM5ES279589,2014,RAM,1500,SLT
1034,2010 Toyota Sequoia Limited,247167.0,Cars By AJ,3.849628,3.0,8995.0,Hot Car,4.8,"['May 30, 2019\nBy Van from Charlotte NC\nOwns...",At least 1 accident or damage reported,...,Four-wheel Drive,13.0,E85 Flex Fuel,Automatic 6-Speed,"I-FORCE 5.7L V-8 DOHC, variable valve control,...",5TDJY5G12AS039485,2010,Toyota,Sequoia,Limited
1163,2004 Nissan Quest 3.5 SE,123397.0,Brick City Motors,3.000000,85.0,4995.0,Hot Car,3.0,"['April 18, 2020\nBy Navigator from Long Beach...",None reported,...,Front-wheel Drive,18.0,Gasoline,5-Speed Automatic,3.5L V6 24V MPFI DOHC,5N1BV28U04N331601,2004,Nissan,Quest,3.5 SE
1620,2018 Nissan Leaf SV,66660.0,GTR Auto Solutions,3.849628,1.0,14900.0,Hot Car,4.6,"['April 2, 2022\nBy DAVE BLAKMON from SEABROO...",None reported,...,Front-wheel Drive,25.4,Electricity,Automatic,Electric,1N4AZ1CP3JC307788,2018,Nissan,Leaf,SV
1902,2007 Chevrolet Monte Carlo SS,115127.0,Brick City Motors,3.000000,85.0,9995.0,Hot Car,3.0,"['September 14, 2022\nBy Demetreis from Chica...",None reported,...,Front-wheel Drive,18.0,Gasoline,4-Speed Automatic,5.3L V8 16V MPFI OHV,2G1WL15C379210269,2007,Chevrolet,Monte,Carlo SS
2128,2011 Ford Explorer XLT,193440.0,Vandross Auto Sales,3.849628,108.0,7900.0,Hot Car,4.3,"['September 19, 2022\nBy Tony from St,Louis M...",None reported,...,Four-wheel Drive,17.0,Gasoline,"Automatic, 6-Spd w/Overdrive","3.5L V-6 DOHC, variable valve control, regular...",1FMHK8D84BGA81604,2011,Ford,Explorer,XLT
2211,2003 Ford Explorer Sport Trac XLS,153914.0,MJ Traders,3.849628,72.0,6995.0,Hot Car,4.7,"['November 14, 2020\nBy Gary from Phoenix\nOwn...",None reported,...,Four-wheel Drive,15.0,Gasoline,5-Speed Automatic,4.0L V6 12V MPFI SOHC,1FMDU77EX3UC81586,2003,Ford,Explorer,Sport Trac XLS
2551,2002 Honda Odyssey EX-L,231396.0,Price Wise Auto Sale,3.849628,0.0,3995.0,Hot Car,4.4,"['June 2, 2022\nBy Oldfridayrider from Davis...",None reported,...,Front-wheel Drive,18.0,Gasoline,Automatic,"3.5L V-6 VTEC variable valve control, regular ...",2HKRL18092H514105,2002,Honda,Odyssey,EX-L
2985,2008 Ford Edge SEL,178296.0,Vandross Auto Sales,3.849628,108.0,5995.0,Hot Car,4.5,"['January 5, 2023\nBy Bunny from Sunrise, F...",At least 1 accident or damage reported,...,Front-wheel Drive,16.0,Gasoline,"Automatic, 6-Spd w/Overdrive","Duratec 3.5L V-6 DOHC, variable valve control,...",2FMDK38C48BA35125,2008,Ford,Edge,SEL


- could be high selling car idk

In [67]:
df

,Car Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Badge Label,Car Rating,Car Reviews,Accidents or damage,...,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Year,Make,Model,Type
0,2016 Audi A4 2.0T Premium,92124.0,NJ State Auto Used Cars,4.200000,672.0,14359.0,Good Deal,4.2,"['April 22, 2021\nBy Tommy from San Antonio tx...",None reported,...,All-wheel Drive,21.0,E85 Flex Fuel,8-Speed Automatic,2.0L I4 16V GDI DOHC Turbo Flexible Fuel,WAUBFAFLXGN009815,2016,Audi,A4,2.0T Premium
1,2015 Buick Enclave Leather,111488.0,NJ State Auto Used Cars,4.200000,672.0,13517.0,Fair Deal,4.2,"['March 26, 2021\nBy Leo Appreciates a great r...",At least 1 accident or damage reported,...,Front-wheel Drive,17.0,Gasoline,6-Speed Automatic,3.6L V6 24V GDI DOHC,5GAKRBKD6FJ125566,2015,Buick,Enclave,Leather
2,2012 Jeep Grand Cherokee Laredo,133821.0,Road Masters II Inc.,3.849628,77.0,9995.0,Great Deal,4.5,"['June 28, 2023\nBy CatTayl from Chatsworth, I...",None reported,...,Four-wheel Drive,16.0,E85 Flex Fuel,Automatic,"3.6L V-6 DOHC, variable valve control, engine ...",1C4RJFAGXCC127988,2012,Jeep,Grand,Cherokee Laredo
3,2010 Toyota Camry LE,116643.0,Arrow Motors,4.800000,220.0,9999.0,Good Deal,5.0,"['January 24, 2023\nBy Pauline from St bernard...",At least 1 accident or damage reported,...,Front-wheel Drive,22.0,Gasoline,6-Speed Automatic,2.5L I4 16V MPFI DOHC,4T1BF3EK9AU561578,2010,Toyota,Camry,LE
4,2016 Mazda Mazda6 i Touring,111588.0,Arrow Motors,4.800000,220.0,13499.0,Good Deal,5.0,"['October 30, 2022\nBy RChico456 from Las Vega...",At least 1 accident or damage reported,...,Front-wheel Drive,25.0,Gasoline,6-Speed Manual,2.5L I4 16V GDI DOHC,JM1GJ1V52G1488572,2016,Mazda,Mazda6,i Touring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9802,2021 Mitsubishi Mirage ES,15035.0,CarVision of Maple Shade,3.849628,0.0,13999.0,Fair Deal,4.9,"['April 18, 2021\nBy LackeyMO from Neosho, Mis...",At least 1 accident or damage reported,...,Front-wheel Drive,33.0,Gasoline,5-Speed Manual,1.2L I3 12V MPFI DOHC,ML32AUHJ7MH010688,2021,Mitsubishi,Mirage,ES
9803,2006 Toyota Corolla S,154352.0,Bethlehem Ford,3.849628,0.0,6524.0,Great Deal,5.0,"['April 27, 2023\nBy Kc from Whittier, MA\nUse...",At least 1 accident or damage reported,...,Front-wheel Drive,30.0,Gasoline,4-Speed Automatic,1.8L I4 16V MPFI DOHC,2T1BR32E16C696228,2006,Toyota,Corolla,S
9804,2008 Suzuki SX4 Base (A4),46460.0,Napoli Kia,3.849628,0.0,8924.0,Great Deal,4.9,"['July 2, 2023\nBy Lee from Dayton ohio \nOwn...",None reported,...,All-wheel Drive,21.0,Gasoline,4-Speed Automatic,2.0L I4 16V MPFI DOHC,JS2YB413385100353,2008,Suzuki,SX4,Base (A4)
9805,2014 Hyundai Santa Fe Sport 2.0L Turbo,152296.0,Pine Belt Chevy of Lakewood,3.849628,0.0,8643.0,Good Deal,5.0,"['June 10, 2021\nBy Adelaide from NC\nOwns thi...",At least 1 accident or damage reported,...,Front-wheel Drive,19.0,Gasoline,6-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,5XYZU3LA5EG187545,2014,Hyundai,Santa,Fe Sport 2.0L Turbo


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9802 entries, 0 to 9806
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Car Name             9802 non-null   object 
 1   Mileage              9802 non-null   float64
 2   Dealer Name          9802 non-null   object 
 3   Dealer Rating        9802 non-null   float64
 4   Review Count         9802 non-null   float64
 5   Price                9802 non-null   float64
 6   Badge Label          9802 non-null   object 
 7   Car Rating           9802 non-null   float64
 8   Car Reviews          9802 non-null   object 
 9   Accidents or damage  9802 non-null   object 
 10  1-owner vehicle      9802 non-null   object 
 11  Personal use only    9802 non-null   object 
 12  Open recall          9802 non-null   object 
 13  Exterior color       9802 non-null   object 
 14  Interior color       9802 non-null   object 
 15  Drivetrain           9802 non-null   o

In [69]:
df.to_csv('/Users/mahmoud/Car_Price_Prediction/working_dataset.csv')